In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import geojson as gj

In [2]:
# Create DF for LAPD_Divisions_geojson
lapd_file = "../static/data/LAPD_Divisions.geojson"
lapd_df = gpd.read_file(lapd_file)

In [14]:
# lapd_df.head()

,FID,APREC,PREC,AREA,PERIMETER,SHAPE_Length,SHAPE_Area,geometry
0,1,MISSION,19,6.995826e+08,190817.495739,0.576177,0.006363,"MULTIPOLYGON (((-118.50736 34.33464, -118.5038..."
1,2,DEVONSHIRE,17,1.346816e+09,175591.702871,0.534007,0.012249,"MULTIPOLYGON (((-118.50736 34.33464, -118.5066..."
2,3,FOOTHILL,16,1.297385e+09,203717.879169,0.627755,0.011797,"MULTIPOLYGON (((-118.41581 34.29384, -118.4156..."
3,4,TOPANGA,21,9.090326e+08,155505.795458,0.470165,0.008260,"MULTIPOLYGON (((-118.63166 34.23858, -118.6315..."
4,5,WEST VALLEY,10,9.364057e+08,136316.899694,0.407795,0.008507,"MULTIPOLYGON (((-118.56229 34.22035, -118.5618..."


In [8]:
# Load json to dataframe
year = [2013,2014,2015,2016,2017,2018,2019]

yearArea_df = pd.DataFrame(columns = ['Year', 'LAPD_Area', 'Crime_Counts'])

# Loop through each year and append to a single dataframe

for y in year:
    crime_year = f"crime_{y}"
    file_to_load = f'../static/data/{crime_year}.json'
    
    crime_df = pd.read_json(file_to_load).T

    # Aggregate by Year and Area
    grouped_df = crime_df.groupby(['Year','LAPD_Area']).agg({'Crime_Type':['count']}) 
    grouped_df.columns = ['Crime_Counts'] 
    grouped_df.reset_index(level=['Year','LAPD_Area'],inplace=True)
    yearArea_df = yearArea_df.append(grouped_df, ignore_index = True)
    

In [9]:
# yearArea_df.head()

In [10]:
# Join grouped_df (crime) with LAPD_divsions - join by area
final_df = yearArea_df.merge(lapd_df, left_on='LAPD_Area', right_on='PREC')
# final_df.head(100)

,Year,LAPD_Area,Crime_Counts,FID,APREC,PREC,AREA,PERIMETER,SHAPE_Length,SHAPE_Area,geometry
0,2013,1,6330,21,CENTRAL,1,1.367477e+08,64187.406589,0.190474,0.001240,"MULTIPOLYGON (((-118.25196 34.07419, -118.2518..."
1,2014,1,6987,21,CENTRAL,1,1.367477e+08,64187.406589,0.190474,0.001240,"MULTIPOLYGON (((-118.25196 34.07419, -118.2518..."
2,2015,1,8880,21,CENTRAL,1,1.367477e+08,64187.406589,0.190474,0.001240,"MULTIPOLYGON (((-118.25196 34.07419, -118.2518..."
3,2016,1,9279,21,CENTRAL,1,1.367477e+08,64187.406589,0.190474,0.001240,"MULTIPOLYGON (((-118.25196 34.07419, -118.2518..."
4,2017,1,10306,21,CENTRAL,1,1.367477e+08,64187.406589,0.190474,0.001240,"MULTIPOLYGON (((-118.25196 34.07419, -118.2518..."
...,...,...,...,...,...,...,...,...,...,...,...
95,2017,14,10006,17,PACIFIC,14,7.176129e+08,246934.321606,0.757182,0.006504,"MULTIPOLYGON (((-118.42224 33.91633, -118.4275..."
96,2018,14,10291,17,PACIFIC,14,7.176129e+08,246934.321606,0.757182,0.006504,"MULTIPOLYGON (((-118.42224 33.91633, -118.4275..."
97,2019,14,10496,17,PACIFIC,14,7.176129e+08,246934.321606,0.757182,0.006504,"MULTIPOLYGON (((-118.42224 33.91633, -118.4275..."
98,2013,15,8220,6,NORTH HOLLYWOOD,15,6.346011e+08,187366.252732,0.566582,0.005765,"MULTIPOLYGON (((-118.34264 34.11645, -118.3427..."


In [11]:
# create function to output a geojson format
def data2geojson(df, filename):
    features = []
    insert_features = lambda X: features.append(
            gj.Feature(geometry=X["geometry"],
                       #gj.Point((X["LON"],X["LAT"])),  ## old code and no longer needed
                            properties=dict(crime_counts=X["Crime_Counts"])))
    df.apply(insert_features, axis=1)
    with open(filename, 'w', encoding='utf8') as fp:
        gj.dump(gj.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)

In [13]:
# Create final crime GeoJson
outputFilePath = '../static/data/'
outputFileName = 'crime'
dispFileName = f'{outputFilePath}{outputFileName}.geojson'

data2geojson(final_df, dispFileName)